<a href="https://colab.research.google.com/github/pruthviraz/pru-electronics/blob/main/pru_electronics.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install streamlit pyngrok

In [ ]:
%%writefile pruthvi_electronics_app.py
import streamlit as st
import sqlite3
import os
import re
from datetime import datetime
import random
import string

# Set the page title and favicon
st.set_page_config(
    page_title="Pruthvi Electrics",
    page_icon="⚡",
    layout="wide"
)

#===============================================================================
# Database setup
#===============================================================================

def init_db():
    os.makedirs('data', exist_ok=True)

    conn = sqlite3.connect('data/pruthvi_electronics.db')
    c = conn.cursor()

    # Products table
    c.execute('''
        CREATE TABLE IF NOT EXISTS products (
            id INTEGER PRIMARY KEY,
            category TEXT NOT NULL,
            name TEXT NOT NULL,
            description TEXT NOT NULL,
            price REAL NOT NULL,
            rating REAL NOT NULL,
            stock INTEGER NOT NULL,
            image_url TEXT
        )
    ''')

    # Orders table
    c.execute('''
        CREATE TABLE IF NOT EXISTS orders (
            id INTEGER PRIMARY KEY,
            customer_name TEXT NOT NULL,
            customer_email TEXT NOT NULL,
            customer_phone TEXT NOT NULL,
            product_id INTEGER NOT NULL,
            product_name TEXT NOT NULL,
            quantity INTEGER NOT NULL,
            total_price REAL NOT NULL,
            order_date TEXT NOT NULL,
            status TEXT NOT NULL,
            tracking_id TEXT UNIQUE NOT NULL,
            FOREIGN KEY (product_id) REFERENCES products (id)
        )
    ''')

    # Enquiries table
    c.execute('''
        CREATE TABLE IF NOT EXISTS enquiries (
            id INTEGER PRIMARY KEY,
            name TEXT NOT NULL,
            email TEXT NOT NULL,
            phone TEXT NOT NULL,
            subject TEXT NOT NULL,
            message TEXT NOT NULL,
            enquiry_date TEXT NOT NULL,
            status TEXT NOT NULL
        )
    ''')

    # Check if products exist
    c.execute("SELECT COUNT(*) FROM products")
    if c.fetchone()[0] == 0:
        # Add sample products
        products = [
            # Televisions
            (1, "Televisions", "Samsung QLED 4K Smart TV",
             "55-inch QLED display with Quantum HDR, Smart TV with built-in streaming apps",
             899.99, 4.7, 25, "https://example.com/samsung_tv.jpg"),
            (2, "Televisions", "LG OLED C1 Series",
             "65-inch OLED 4K TV with AI ThinQ, Dolby Vision IQ and Dolby Atmos",
             1499.99, 4.8, 15, "https://example.com/lg_tv.jpg"),
            (3, "Televisions", "Sony Bravia X90J",
             "50-inch 4K HDR LED Smart TV with Google TV and Dolby Vision",
             999.99, 4.6, 20, "https://example.com/sony_tv.jpg"),
            (4, "Televisions", "TCL 6-Series Roku TV",
             "55-inch 4K QLED Dolby Vision HDR Smart Roku TV",
             699.99, 4.5, 30, "https://example.com/tcl_tv.jpg"),
            (5, "Televisions", "Hisense ULED U7G",
             "65-inch Quantum Dot 4K ULED Smart Android TV",
             899.99, 4.4, 18, "https://example.com/hisense_tv.jpg"),
            (6, "Televisions", "Samsung The Frame",
             "55-inch 4K QLED Smart TV with Art Mode, Customizable Bezels",
             1299.99, 4.6, 12, ""),
            (7, "Televisions", "LG NanoCell 90 Series",
             "75-inch 4K Smart TV with AI ThinQ, Full Array Dimming",
             1599.99, 4.5, 8, ""),
            (8, "Televisions", "Sony A95K QD-OLED",
             "65-inch 4K QD-OLED TV with Cognitive Processor XR",
             2999.99, 4.9, 5, ""),

            # Laptops
            (9, "Laptops", "Apple MacBook Pro 14-inch",
             "M1 Pro chip, 16GB RAM, 512GB SSD, Liquid Retina XDR display",
             1999.99, 4.9, 12, "https://example.com/macbook.jpg"),
            (10, "Laptops", "Dell XPS 15",
             "Intel Core i7, 16GB RAM, 512GB SSD, 15.6\" 4K OLED Touch Display",
             1699.99, 4.7, 15, "https://example.com/dell_xps.jpg"),
            (11, "Laptops", "HP Spectre x360",
             "Intel Core i7, 16GB RAM, 1TB SSD, 13.5\" 3K2K OLED Touch Display",
             1399.99, 4.6, 10, "https://example.com/hp_spectre.jpg"),
            (12, "Laptops", "Lenovo ThinkPad X1 Carbon",
             "Intel Core i5, 16GB RAM, 512GB SSD, 14\" FHD Display",
             1299.99, 4.5, 18, "https://example.com/thinkpad.jpg"),
            (13, "Laptops", "Asus ROG Zephyrus G14",
             "AMD Ryzen 9, 16GB RAM, 1TB SSD, NVIDIA RTX 3060, 14\" QHD Display",
             1499.99, 4.8, 8, "https://example.com/asus_rog.jpg"),
            (14, "Laptops", "Microsoft Surface Laptop 5",
             "Intel Core i7, 16GB RAM, 512GB SSD, 15\" Touchscreen",
             1299.99, 4.6, 10, ""),
            (15, "Laptops", "Acer Swift X",
             "AMD Ryzen 7, 16GB RAM, 1TB SSD, NVIDIA RTX 3050 Ti, 14\" FHD",
             1099.99, 4.5, 12, ""),
            (16, "Laptops", "Razer Blade 15",
             "Intel Core i7, 32GB RAM, 1TB SSD, NVIDIA RTX 3070 Ti, 15.6\" QHD",
             2499.99, 4.7, 6, ""),

            # Smartphones
            (17, "Smartphones", "iPhone 15 Pro", "6.1-inch Super Retina XDR display, A17 Pro chip, Triple-camera system", 999.99, 4.8, 30, ""),
            (18, "Smartphones", "Samsung Galaxy S23 Ultra", "6.8-inch Dynamic AMOLED, S Pen, 200MP camera", 1199.99, 4.7, 25, ""),
            (19, "Smartphones", "Google Pixel 8 Pro", "6.7-inch LTPO OLED, Google Tensor G3, 50MP main camera", 999.99, 4.6, 20, ""),
            (20, "Smartphones", "OnePlus 11", "6.7-inch Fluid AMOLED, Snapdragon 8 Gen 2, Hasselblad camera", 799.99, 4.5, 18, ""),
            (21, "Smartphones", "Xiaomi 13 Pro", "6.73-inch AMOLED, Snapdragon 8 Gen 2, 50MP triple camera", 999.99, 4.4, 15, ""),
            (22, "Smartphones", "Nothing Phone (2)", "6.7-inch LTPO OLED, Snapdragon 8+ Gen 1, Glyph Interface", 599.99, 4.3, 20, ""),
            (23, "Smartphones", "ASUS ROG Phone 7", "6.78-inch AMOLED, Snapdragon 8 Gen 2, 165Hz refresh rate", 999.99, 4.5, 10, ""),
            (24, "Smartphones", "Sony Xperia 1 V", "6.5-inch 4K OLED, Snapdragon 8 Gen 2, Zeiss optics", 1399.99, 4.4, 8, ""),

            # Headphones
            (25, "Headphones", "Sony WH-1000XM5", "Industry-leading noise cancellation, 30-hour battery", 399.99, 4.8, 40, ""),
            (26, "Headphones", "Bose QuietComfort 45", "Premium noise cancelling, 24-hour battery", 329.99, 4.7, 35, ""),
            (27, "Headphones", "Apple AirPods Max", "Active Noise Cancellation, spatial audio, 20-hour battery", 549.99, 4.6, 25, ""),
            (28, "Headphones", "Sennheiser Momentum 4", "Adaptive Noise Cancellation, 60-hour battery", 399.99, 4.5, 30, ""),
            (29, "Headphones", "Jabra Elite 85h", "SmartSound technology, 36-hour battery", 249.99, 4.4, 20, ""),
            (30, "Headphones", "Bowers & Wilkins Px7 S2", "Adaptive noise cancellation, 30-hour battery", 399.99, 4.6, 15, ""),
            (31, "Headphones", "Beats Studio Pro", "Active Noise Cancelling, 40-hour battery", 349.99, 4.3, 25, ""),
            (32, "Headphones", "Audio-Technica ATH-M50xBT2", "Professional wireless headphones, 50-hour battery", 199.99, 4.5, 20, ""),

            # Smartwatches
            (33, "Smartwatches", "Apple Watch Series 9", "45mm, GPS + Cellular, Always-On Retina display", 529.99, 4.8, 30, ""),
            (34, "Smartwatches", "Samsung Galaxy Watch 6 Classic", "47mm, LTE, Wear OS, Rotating bezel", 429.99, 4.7, 25, ""),
            (35, "Smartwatches", "Garmin Venu 3", "AMOLED display, Advanced sleep monitoring, 14-day battery", 449.99, 4.6, 20, ""),
            (36, "Smartwatches", "Fitbit Sense 2", "Advanced health metrics, ECG app, 6+ day battery", 299.99, 4.5, 35, ""),
            (37, "Smartwatches", "Fossil Gen 7", "Snapdragon Wear 4100+, 24+ hour battery, Wear OS", 299.99, 4.3, 15, ""),
            (38, "Smartwatches", "Garmin Epix Pro", "AMOLED display, Multi-band GPS, 16-day battery", 899.99, 4.7, 10, ""),
            (39, "Smartwatches", "Amazfit GTR 4", "1.43\" AMOLED, 14-day battery, 150+ sports modes", 199.99, 4.4, 25, ""),
            (40, "Smartwatches", "Withings ScanWatch", "Hybrid smartwatch with ECG and SpO2", 299.99, 4.5, 18, ""),

            # Cameras
            (41, "Cameras", "Sony A7 IV", "33MP full-frame mirrorless camera, 4K 60p video", 2499.99, 4.9, 10, ""),
            (42, "Cameras", "Canon EOS R5", "45MP full-frame mirrorless, 8K video, IBIS", 3899.99, 4.8, 8, ""),
            (43, "Cameras", "Nikon Z8", "45MP full-frame mirrorless, 8K video, Advanced AF", 3999.99, 4.9, 6, ""),
            (44, "Cameras", "Fujifilm X-T5", "40MP APS-C, 6.2K video, 7-stop IBIS", 1699.99, 4.7, 15, ""),
            (45, "Cameras", "Panasonic Lumix S5 II", "24MP full-frame, 6K video, Phase Hybrid AF", 1999.99, 4.6, 12, ""),
            (46, "Cameras", "Olympus OM-1", "20MP Micro Four Thirds, Weather-sealed, 50fps shooting", 2199.99, 4.7, 10, ""),
            (47, "Cameras", "Leica Q3", "60MP full-frame, 8K video, Summilux 28mm f/1.7", 5999.99, 4.9, 3, ""),
            (48, "Cameras", "GoPro Hero 12 Black", "5.3K video, HyperSmooth 6.0, HDR", 399.99, 4.6, 25, ""),

            # Gaming Consoles
            (49, "Gaming Consoles", "PlayStation 5", "4K UHD Blu-ray, 825GB SSD, 120fps support", 499.99, 4.9, 5, ""),
            (50, "Gaming Consoles", "Xbox Series X", "4K gaming at 120fps, 1TB SSD, Quick Resume", 499.99, 4.8, 7, ""),
            (51, "Gaming Consoles", "Nintendo Switch OLED", "7-inch OLED screen, 64GB storage, Handheld/tabletop/TV modes", 349.99, 4.7, 20, ""),
            (52, "Gaming Consoles", "Steam Deck (1TB)", "7-inch touchscreen, AMD APU, 1TB NVMe SSD", 649.99, 4.6, 15, ""),
            (53, "Gaming Consoles", "Meta Quest 3", "Mixed reality headset, Snapdragon XR2 Gen 2, 128GB", 499.99, 4.5, 25, ""),
            (54, "Gaming Consoles", "ASUS ROG Ally", "7-inch 120Hz display, AMD Ryzen Z1 Extreme, 512GB", 699.99, 4.7, 12, ""),
            (55, "Gaming Consoles", "PlayStation VR2", "4K HDR OLED, Eye tracking, Haptic feedback", 549.99, 4.6, 18, ""),
            (56, "Gaming Consoles", "Nintendo Switch Pro Controller", "Official controller with HD rumble", 69.99, 4.8, 30, ""),

            # Home Appliances
            (57, "Home Appliances", "Dyson V15 Detect", "Laser dust detection, 60-minute runtime, HEPA filtration", 699.99, 4.8, 15, ""),
            (58, "Home Appliances", "iRobot Roomba j7+", "Self-emptying, Smart mapping, Avoids obstacles", 799.99, 4.7, 12, ""),
            (59, "Home Appliances", "Ninja Foodi 10-in-1", "Pressure cooker, air fryer, slow cooker combo", 199.99, 4.6, 30, ""),
            (60, "Home Appliances", "Vitamix 5200", "Professional-grade blender, 64-oz container", 449.99, 4.8, 20, ""),
            (61, "Home Appliances", "Breville Barista Express", "Espresso machine with built-in grinder", 699.99, 4.7, 10, ""),
            (62, "Home Appliances", "Instant Pot Duo Plus", "9-in-1 pressure cooker, sous vide, yogurt maker", 129.99, 4.6, 25, ""),
            (63, "Home Appliances", "Roborock S8 Pro Ultra", "Self-cleaning, self-emptying robot vacuum", 1299.99, 4.8, 8, ""),
            (64, "Home Appliances", "GE Profile Smart Countertop Oven", "Smart convection oven with air fry", 499.99, 4.5, 15, ""),

            # Audio Systems
            (65, "Audio Systems", "Sonos Arc", "Dolby Atmos soundbar, Voice control compatible", 899.99, 4.8, 15, ""),
            (66, "Audio Systems", "Bose Smart Soundbar 900", "Dolby Atmos, Voice control, WiFi/Bluetooth", 899.99, 4.7, 12, ""),
            (67, "Audio Systems", "Sony HT-A9", "360 Spatial Sound Mapping, 4 wireless speakers", 1799.99, 4.9, 5, ""),
            (68, "Audio Systems", "Samsung HW-Q990C", "11.1.4ch Soundbar with Wireless Subwoofer", 1899.99, 4.8, 8, ""),
            (69, "Audio Systems", "LG S95QR", "9.1.5ch Soundbar with Meridian Technology", 1599.99, 4.7, 10, ""),
            (70, "Audio Systems", "Klipsch Cinema 1200", "5.1.4 Dolby Atmos Soundbar with 12\" subwoofer", 1499.99, 4.7, 12, ""),
            (71, "Audio Systems", "JBL Bar 1000", "7.1.4-channel soundbar with detachable speakers", 1299.99, 4.6, 15, ""),
            (72, "Audio Systems", "Yamaha YAS-209", "Bluetooth soundbar with wireless subwoofer", 399.99, 4.5, 20, ""),

            # Networking
            (73, "Networking", "TP-Link Archer AX6000", "WiFi 6 router, 8 Gigabit LAN ports", 299.99, 4.6, 25, ""),
            (74, "Networking", "Netgear Nighthawk AX12", "WiFi 6 router, 12-stream, 6Gbps speed", 499.99, 4.7, 15, ""),
            (75, "Networking", "Google Nest Wifi Pro", "WiFi 6E mesh system, 2-pack, 4400 sq ft coverage", 399.99, 4.5, 30, ""),
            (76, "Networking", "Eero Pro 6E", "Tri-band mesh WiFi 6E system, 1-pack", 299.99, 4.6, 20, ""),
            (77, "Networking", "ASUS RT-AX86U", "AX5700 Dual Band WiFi 6 Gaming Router", 249.99, 4.7, 18, ""),
            (78, "Networking", "Ubiquiti UniFi Dream Machine", "All-in-one security gateway and WiFi 6 router", 379.99, 4.8, 12, ""),
            (79, "Networking", "Linksys Atlas Max 6E", "Tri-band WiFi 6E mesh system, 2-pack", 499.99, 4.6, 15, ""),
            (80, "Networking", "Synology RT6600ax", "Tri-band WiFi 6 router, 5.9GHz band", 299.99, 4.7, 10, ""),

            # Printers & Scanners
            (81, "Printers & Scanners", "Epson EcoTank ET-4850", "Wireless all-in-one printer with refillable ink tanks", 399.99, 4.7, 20, ""),
            (82, "Printers & Scanners", "HP OfficeJet Pro 9025e", "All-in-one wireless printer with smart features", 299.99, 4.6, 25, ""),
            (83, "Printers & Scanners", "Brother HL-L2395DW", "Monochrome laser printer with wireless networking", 199.99, 4.5, 30, ""),
            (84, "Printers & Scanners", "Canon PIXMA TR8620a", "All-in-one wireless photo printer with auto document feeder", 249.99, 4.4, 18, ""),
            (85, "Printers & Scanners", "Epson WorkForce Pro WF-4820", "Wireless all-in-one printer with ADF", 199.99, 4.5, 15, ""),
            (86, "Printers & Scanners", "HP LaserJet Pro MFP M283fdw", "Color laser all-in-one with wireless and fax", 399.99, 4.6, 12, ""),
            (87, "Printers & Scanners", "Brother MFC-J995DW", "Inkjet all-in-one with wide format printing", 299.99, 4.5, 20, ""),
            (88, "Printers & Scanners", "Fujitsu ScanSnap iX1500", "Document scanner with Wi-Fi and cloud integration", 499.99, 4.7, 10, ""),

            # Wearable Tech
            (89, "Wearable Tech", "Oura Ring (Gen 3)", "Sleep and activity tracking smart ring", 299.99, 4.6, 25, ""),
            (90, "Wearable Tech", "Whoop Strap 4.0", "24/7 fitness and recovery tracker", 299.99, 4.5, 20, ""),
            (91, "Wearable Tech", "Fitbit Charge 6", "Advanced fitness tracker with Google apps", 159.99, 4.4, 30, ""),
            (92, "Wearable Tech", "Garmin Vivosmart 5", "Health and fitness tracker with Pulse Ox", 149.99, 4.3, 25, ""),
            (93, "Wearable Tech", "Amazfit Band 7", "1.47\" AMOLED display, 18-day battery life", 59.99, 4.2, 35, ""),
            (94, "Wearable Tech", "Polar H10", "Heart rate sensor chest strap", 89.99, 4.7, 15, ""),
            (95, "Wearable Tech", "Coros Pace 3", "GPS running watch with multi-sport tracking", 229.99, 4.5, 18, ""),
            (96, "Wearable Tech", "Suunto 9 Peak Pro", "Premium GPS sports watch with long battery", 599.99, 4.6, 10, ""),

            # Drones
            (97, "Drones", "DJI Mavic 3 Pro", "4/3 CMOS Hasselblad camera, 43-minute flight time", 2199.99, 4.9, 5, ""),
            (98, "Drones", "DJI Mini 4 Pro", "Ultralight 249g drone with 4K HDR video", 759.99, 4.8, 15, ""),
            (99, "Drones", "Autel Robotics EVO Lite+", "1-inch CMOS sensor, 6K camera, 40-minute flight", 1199.99, 4.7, 8, ""),
            (100, "Drones", "DJI Air 3", "Dual camera with 3x optical zoom, 46-minute flight", 1099.99, 4.8, 10, ""),
            (101, "Drones", "Skydio 2+", "Autonomous drone with 4K60 HDR video", 1099.99, 4.7, 6, ""),
            (102, "Drones", "DJI Avata", "FPV drone with 4K stabilized video", 999.99, 4.6, 12, ""),
            (103, "Drones", "Parrot Anafi", "Compact 4K HDR drone with 180° tilt gimbal", 699.99, 4.5, 15, ""),
            (104, "Drones", "Holy Stone HS720G", "GPS drone with 4K camera and 2 batteries", 299.99, 4.3, 20, ""),

            # Smart Home
            (105, "Smart Home", "Nest Learning Thermostat", "3rd gen smart thermostat that programs itself", 249.99, 4.7, 25, ""),
            (106, "Smart Home", "Ring Video Doorbell Pro 2", "HD video doorbell with 3D motion detection", 249.99, 4.6, 20, ""),
            (107, "Smart Home", "Arlo Pro 4", "Wire-free 2K HDR security camera with spotlight", 199.99, 4.5, 18, ""),
            (108, "Smart Home", "Ecobee Smart Thermostat Premium", "Smart thermostat with built-in Alexa and air quality", 249.99, 4.6, 15, ""),
            (109, "Smart Home", "August Wi-Fi Smart Lock", "Keyless entry that works with Alexa and Google", 229.99, 4.5, 20, ""),
            (110, "Smart Home", "Philips Hue White and Color Ambiance", "Smart LED bulbs with 16 million colors", 199.99, 4.7, 30, ""),
            (111, "Smart Home", "Google Nest Hub (2nd Gen)", "7\" smart display with sleep sensing", 99.99, 4.5, 25, ""),
            (112, "Smart Home", "Eufy RoboVac G30", "Robot vacuum with smart navigation", 299.99, 4.4, 15, ""),
        ]

        c.executemany("INSERT INTO products VALUES (?, ?, ?, ?, ?, ?, ?, ?)", products)
        conn.commit()

    conn.close()

init_db()

#===============================================================================
# Database operations
#===============================================================================

def get_products_by_category(category):
    conn = sqlite3.connect('data/pruthvi_electronics.db')
    conn.row_factory = sqlite3.Row
    c = conn.cursor()
    c.execute("SELECT * FROM products WHERE category=? ORDER BY name", (category,))
    result = [dict(row) for row in c.fetchall()]
    conn.close()
    return result

def get_product_by_id(product_id):
    conn = sqlite3.connect('data/pruthvi_electronics.db')
    conn.row_factory = sqlite3.Row
    c = conn.cursor()
    c.execute("SELECT * FROM products WHERE id=?", (product_id,))
    result = c.fetchone()
    conn.close()
    return dict(result) if result else None

def get_all_categories():
    conn = sqlite3.connect('data/pruthvi_electronics.db')
    c = conn.cursor()
    c.execute("SELECT DISTINCT category FROM products ORDER BY category")
    result = [row[0] for row in c.fetchall()]
    conn.close()
    return result

def search_products(query):
    conn = sqlite3.connect('data/pruthvi_electronics.db')
    conn.row_factory = sqlite3.Row
    c = conn.cursor()
    c.execute("SELECT * FROM products WHERE name LIKE ? OR description LIKE ? ORDER BY name",
              (f"%{query}%", f"%{query}%"))
    result = [dict(row) for row in c.fetchall()]
    conn.close()
    return result

def generate_tracking_id():
    """Generate a unique tracking ID with format PRU-YYYYMMDD-XXXXXX"""
    date_part = datetime.now().strftime('%Y%m%d')
    random_part = ''.join(random.choices(string.ascii_uppercase + string.digits, k=6))
    return f"PRU-{date_part}-{random_part}"

def create_order(order_data):
    conn = sqlite3.connect('data/pruthvi_electronics.db')
    c = conn.cursor()

    # Get product price
    c.execute("SELECT price FROM products WHERE id=?", (order_data['product_id'],))
    product_price = c.fetchone()[0]
    total_price = product_price * order_data['quantity']

    # Generate tracking ID
    tracking_id = generate_tracking_id()

    try:
        c.execute(
            """INSERT INTO orders
            (customer_name, customer_email, customer_phone, product_id, product_name,
             quantity, total_price, order_date, status, tracking_id)
            VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?)""",
            (
                order_data['customer_name'],
                order_data['customer_email'],
                order_data['customer_phone'],
                order_data['product_id'],
                order_data['product_name'],
                order_data['quantity'],
                total_price,
                datetime.now().strftime('%Y-%m-%d %H:%M:%S'),
                "Processing",
                tracking_id
            )
        )

        # Update product stock
        c.execute("UPDATE products SET stock = stock - ? WHERE id = ?",
                  (order_data['quantity'], order_data['product_id']))

        conn.commit()
        return tracking_id
    except sqlite3.Error as e:
        conn.rollback()
        raise e
    finally:
        conn.close()

def get_order_by_tracking(tracking_id):
    conn = sqlite3.connect('data/pruthvi_electronics.db')
    conn.row_factory = sqlite3.Row
    c = conn.cursor()
    c.execute("SELECT * FROM orders WHERE tracking_id=?", (tracking_id,))
    result = c.fetchone()
    conn.close()
    return dict(result) if result else None

def create_enquiry(enquiry_data):
    conn = sqlite3.connect('data/pruthvi_electronics.db')
    c = conn.cursor()

    try:
        c.execute(
            """INSERT INTO enquiries
            (name, email, phone, subject, message, enquiry_date, status)
            VALUES (?, ?, ?, ?, ?, ?, ?)""",
            (
                enquiry_data['name'],
                enquiry_data['email'],
                enquiry_data['phone'],
                enquiry_data['subject'],
                enquiry_data['message'],
                datetime.now().strftime('%Y-%m-%d %H:%M:%S'),
                "New"
            )
        )
        conn.commit()
    except sqlite3.Error as e:
        conn.rollback()
        raise e
    finally:
        conn.close()

#===============================================================================
# Helper functions
#===============================================================================

def validate_email(email):
    pattern = r'^[\w\.-]+@[\w\.-]+\.[a-z]{2,}$'
    return re.match(pattern, email) is not None

def validate_phone(phone):
    pattern = r'^\+?\d{10,15}$'
    return re.match(pattern, phone) is not None

def add_message(role, message):
    st.session_state.chat_history.append({"role": role, "message": message})

def reset_chat():
    st.session_state.chat_history = []
    st.session_state.current_step = 'greeting'
    st.session_state.order_data = {}
    add_message("assistant", "👋 Welcome to Pruthvi Electrics! How can I assist you today?")

def display_order_confirmation(tracking_id):
    """Display order confirmation with tracking information"""
    order = get_order_by_tracking(tracking_id)
    if order:
        st.success("🎉 Your order has been placed successfully!")
        st.markdown(f"""
        **Order Summary:**
        - **Product:** {order['product_name']}
        - **Quantity:** {order['quantity']}
        - **Total Price:** ₹{order['total_price']:.2f}
        - **Order Date:** {order['order_date']}
        - **Status:** {order['status']}
        - **Tracking ID:** `{order['tracking_id']}`

        We've sent a confirmation to your email. You can track your order using the tracking ID above.
        """)
    else:
        st.error("We couldn't retrieve your order details. Please contact customer support.")

# Initialize session state
if 'chat_history' not in st.session_state:
    st.session_state.chat_history = []

if 'current_step' not in st.session_state:
    st.session_state.current_step = 'greeting'

if 'order_data' not in st.session_state:
    st.session_state.order_data = {}

if 'selected_product' not in st.session_state:
    st.session_state.selected_product = None

if 'selected_category' not in st.session_state:
    st.session_state.selected_category = None

if 'order_confirmation' not in st.session_state:
    st.session_state.order_confirmation = None

if len(st.session_state.chat_history) == 0:
    reset_chat()

#===============================================================================
# UI Layout
#===============================================================================

# Sidebar - Company Information
with st.sidebar:
    st.title("Pruthvi Electrics")
    st.subheader("Your Trusted Electronics Partner")

    st.markdown("""
    **Address:**
    123 Electronics Street
    Bengaluru, Karnataka 560001
    India

    **Contact:**
    ☎️ +91 80 1234 5678
    📧 info@pruthvielectrics.com

    **Business Hours:**
    Monday - Saturday: 9:00 AM - 9:00 PM
    Sunday: 10:00 AM - 6:00 PM
    """)

    st.markdown("---")
    st.subheader("Quick Enquiry")

    with st.form("enquiry_form"):
        name = st.text_input("Your Name")
        email = st.text_input("Email")
        phone = st.text_input("Phone")
        subject = st.selectbox("Subject", ["Product Inquiry", "Order Status", "Complaint", "Other"])
        message = st.text_area("Your Message")

        if st.form_submit_button("Submit Enquiry"):
            if name and email and phone and message:
                if validate_email(email) and validate_phone(phone):
                    enquiry_data = {
                        'name': name,
                        'email': email,
                        'phone': phone,
                        'subject': subject,
                        'message': message
                    }
                    try:
                        create_enquiry(enquiry_data)
                        st.success("Thank you! Your enquiry has been submitted. We'll contact you soon.")
                    except Exception as e:
                        st.error(f"Error submitting enquiry: {e}")
                else:
                    st.error("Please enter valid email and phone number.")
            else:
                st.error("Please fill all required fields.")

# Main content area
col1, col2 = st.columns([1, 3])

# Left column - Categories
with col1:
    st.subheader("Product Categories")
    categories = get_all_categories()

    for category in categories:
        if st.button(category):
            st.session_state.selected_category = category
            st.session_state.selected_product = None
            st.session_state.order_confirmation = None
            st.rerun()

    st.markdown("---")
    st.subheader("Order Tracking")

    tracking_id = st.text_input("Enter Tracking ID", key="tracking_input")
    if st.button("Track Order", key="track_button"):
        if tracking_id:
            order = get_order_by_tracking(tracking_id)
            if order:
                st.success(f"**Order Status:** {order['status']}")
                st.markdown(f"""
                - **Product:** {order['product_name']}
                - **Quantity:** {order['quantity']}
                - **Total Price:** ₹{order['total_price']:.2f}
                - **Order Date:** {order['order_date']}
                """)
            else:
                st.error("Order not found. Please check your tracking ID.")
        else:
            st.warning("Please enter a tracking ID")

# Right column - Main content
with col2:
    st.title("Pruthvi Electrics E-Commerce")

    # Show order confirmation if available
    if st.session_state.order_confirmation:
        display_order_confirmation(st.session_state.order_confirmation)
        if st.button("Continue Shopping"):
            st.session_state.order_confirmation = None
            st.session_state.selected_product = None
            st.session_state.selected_category = None
            st.rerun()
    else:
        # Search bar
        search_query = st.text_input("Search products...", key="search")
        if search_query:
            search_results = search_products(search_query)
            if search_results:
                st.subheader("Search Results")
                for product in search_results:
                    if st.button(f"{product['name']} - ₹{product['price']:.2f}", key=f"search_{product['id']}"):
                        st.session_state.selected_product = product
                        st.session_state.selected_category = product['category']
                        st.rerun()
            else:
                st.warning("No products found matching your search.")

        # Display selected category products or selected product
        if st.session_state.selected_product:
            product = st.session_state.selected_product
            st.subheader(product['name'])

            col_img, col_info = st.columns([1, 2])

            with col_img:
                if product['image_url']:
                    st.image(product['image_url'], width=200)
                else:
                    st.image("https://via.placeholder.com/200", width=200)

            with col_info:
                st.markdown(f"**Category:** {product['category']}")
                st.markdown(f"**Price:** ₹{product['price']:.2f}")
                st.markdown(f"**Rating:** {product['rating']} ⭐")
                st.markdown(f"**In Stock:** {product['stock']}")
                st.markdown("**Description:**")
                st.write(product['description'])

                # Order form
                with st.form(f"order_form_{product['id']}"):
                    st.write("Place Order")
                    name = st.text_input("Your Name", key=f"name_{product['id']}")
                    email = st.text_input("Email", key=f"email_{product['id']}")
                    phone = st.text_input("Phone", key=f"phone_{product['id']}")
                    quantity = st.number_input("Quantity", min_value=1, max_value=product['stock'], value=1,
                                            key=f"quantity_{product['id']}")

                    if st.form_submit_button("Place Order"):
                        if name and email and phone:
                            if validate_email(email) and validate_phone(phone):
                                order_data = {
                                    'customer_name': name,
                                    'customer_email': email,
                                    'customer_phone': phone,
                                    'product_id': product['id'],
                                    'product_name': product['name'],
                                    'quantity': quantity
                                }
                                try:
                                    tracking_id = create_order(order_data)
                                    st.session_state.order_confirmation = tracking_id
                                    st.rerun()
                                except Exception as e:
                                    st.error(f"Error placing order: {e}")
                            else:
                                st.error("Please enter valid email and phone number.")
                        else:
                            st.error("Please fill all required fields.")

        elif st.session_state.selected_category:
            st.subheader(st.session_state.selected_category)
            products = get_products_by_category(st.session_state.selected_category)

            for product in products:
                if st.button(f"{product['name']} - ₹{product['price']:.2f}", key=f"product_{product['id']}"):
                    st.session_state.selected_product = product
                    st.rerun()

        # Chatbot interface
        st.markdown("---")
        st.subheader("Chat with Our Assistant")

        # Display chat history
        for message in st.session_state.chat_history:
            if message["role"] == "user":
                st.chat_message("user").write(message["message"])
            else:
                st.chat_message("assistant").write(message["message"])

        # Process user input
        user_input = st.chat_input("Type your message here...")
        if user_input:
            add_message("user", user_input)

            # Enhanced chatbot responses
            user_input_lower = user_input.lower()

            if any(word in user_input_lower for word in ["order", "buy", "purchase"]):
                response = "Please select a category from the left to browse our products. You can also use the search bar to find specific items."
            elif any(word in user_input_lower for word in ["track", "status", "where is my order"]):
                response = "You can check your order status by entering your tracking ID in the 'Order Tracking' section on the left. Your tracking ID was provided when you placed the order."
            elif any(word in user_input_lower for word in ["help", "support", "problem"]):
                response = "You can submit an enquiry using the form on the left, or ask me specific questions about our products. For urgent issues, please call our support line at +91 80 1234 5678."
            elif any(word in user_input_lower for word in ["thank", "thanks"]):
                response = "You're welcome! Is there anything else I can help you with?"
            elif any(word in user_input_lower for word in ["hi", "hello", "hey"]):
                response = "Hello! Welcome to Pruthvi Electrics. How can I assist you today?"
            elif any(word in user_input_lower for word in ["price", "cost", "how much"]):
                response = "Product prices are displayed with each item. Please select a category to view our products and their prices."
            elif any(word in user_input_lower for word in ["delivery", "shipping"]):
                response = "We offer standard shipping (3-5 business days) and express shipping (1-2 business days). Shipping costs vary based on your location."
            elif any(word in user_input_lower for word in ["return", "exchange"]):
                response = "We have a 30-day return policy for most items. Please contact our support team for return or exchange requests."
            else:
                response = "I can help you with product information, orders, and tracking. How can I assist you?"

            add_message("assistant", response)
            st.rerun()

# Reset button
if st.button("Reset Chat", key="reset_chat"):
    reset_chat()
    st.rerun()

Overwriting pruthvi_electronics_app.py


In [ ]:
from pyngrok import ngrok
import os

# Set your authtoken
ngrok.set_auth_token("2z0yjbxYZ7DgOiV4HJwXD2UCf81_2Bpw1Py7tJEVo9tzmzA4r")

# Kill any existing ngrok processes
ngrok.kill()

# Open a tunnel to port 8501 (Streamlit's default port)
public_url = ngrok.connect(8501)
print(f"Streamlit app is available at: {public_url}")

# Run the Streamlit app
!streamlit run pruthvi_electronics_app.py --server.port=8501 &

# Wait for app to start
import time
time.sleep(5)  # Wait 5 seconds for the app to start

Streamlit app is available at: NgrokTunnel: "https://7559-34-138-56-160.ngrok-free.app" -> "http://localhost:8501"



  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8501
  Network URL: http://172.28.0.12:8501
  External URL: http://34.138.56.160:8501

  Stopping...
